# How to run pygrass `Modules` in parallel

It is possible to run `Modules` object in parallel. There are more ways, in this documents there are two of them.

* using `subprocess` module
* using `ParallelModuleQueue` pygrass modules

## subprocess

`subprocess` is the Python module to spawn new processes, connect to their input/output/error pipes, and obtain their return codes.

In [ ]:
from multiprocessing import Queue, Process
import grass.script as grass

First it is required to create a function with the analysis needed, in this case create a buffer around the input vector map.

In [ ]:
def calculate(inp, dist):
    # function to create a buffer
    out = "{ip}_buffer_{di}".format(ip=inp, di=dist)
    grass.run_command("v.buffer", input=inp, output=out, distance=dist, overwrite=True, quiet=True)
    return out

In [ ]:
def spawn(func):
    def funct(q_in, q_out):
        while True:
            vec, dis = q_in.get()
            if vec is None:
                break
            q_out.put(func(vec, dis))
    return funct

Now create the needed input

In [ ]:
# two queue object, one for input and output
q_in = Queue(1)
q_out = Queue()
# number of process to create
nproc = 3
# input vector and distances
invect = "schools"
inbuffer = range(0,220,20)

In [ ]:
Queue?

Create a list of process according to `nproc` variable, the target is the `spawn` function with `calculate` as variable for `spawn`

In [ ]:
procs = [Process(target=spawn(calculate), args=(q_in, q_out)) for _ in range(nproc)]

In [ ]:
Process?

For each process set it as daemon and start the process

In [ ]:
for proc in procs:
    proc.daemon = True
    proc.start()

Set the inputs (vector map and distance)

In [ ]:
ans = [q_in.put((invect, i)) for i in inbuffer if i > 0]

Set `None` to each processes to terminate them

In [ ]:
[q_in.put((None, None)) for proc in procs]

At this point check if the processes run correctly and if the maps are created

In [ ]:
[proc.join() for proc in procs]
processed = [q_out.get() for _ in ans]
if len(processed) != len(inbuffer) - 1:
    print("An error occurs")
print(grass.read_command('g.list', type="vector", pattern="schools_buffer*", mapset='.'))

## ParallelModuleQueue

The `ParallelModuleQueue` is designed to run an arbitrary number of pygrass Module or MultiModule processes in parallel. 

In [ ]:
from grass.pygrass.modules import Module, ParallelModuleQueue

In [ ]:
ParallelModuleQueue?

Initialize the `ParallelModuleQueue` object

In [ ]:
queue = ParallelModuleQueue(nprocs=nproc)

Put the `Modules` objects inside the `queue` and wait until the finish

In [ ]:
val = 0
while val < 200:
    queue.put(Module("r.mapcalc",
                     expression="elevation_{va}=if(elevation@PERMANENT>{va} && elevation@PERMANENT<={va2}, elevation@PERMANENT, null())".format(va=val, va2=val+50),
                     overwrite=True, run_=False, finish_=False)
             )
    val += 50
queue.wait()

Check how the processes terminate

In [ ]:
print("Number of running process ".format(queue.get_num_run_procs()))
print("Number of max number of process ".format(queue.get_max_num_procs()))

In [ ]:
for mapcalc in queue.get_finished_modules():
    print(mapcalc.popen.returncode)

# Summary

We have seen

* how to run GRASS modules using the standard library `subprocess` module
* how to use `ParallelModuleQueue` pygrass module

# Exercise